# LangGraph Stream Parser - Jupyter Display Demo

This notebook demonstrates the value of `JupyterDisplay` for visualizing LangGraph agent streams.

## The Problem

When streaming from a LangGraph agent, the raw output is:
- **Verbose** - Nested dictionaries with metadata you don't care about
- **Hard to follow** - Tool calls, messages, and interrupts all mixed together
- **Not real-time friendly** - Output floods the cell instead of updating in place

## The Solution

`JupyterDisplay` provides:
- **Clean panels** - Content, tools, and interrupts in organized boxes
- **Live updates** - Updates in place instead of flooding output
- **Tool lifecycle** - See tools transition from running → success/error with timing
- **Prominent interrupts** - Human-in-the-loop requests are impossible to miss

## Setup

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from agent import agent

---
## Before: Raw Streaming Output

This is what you get with vanilla `graph.stream()` - a flood of nested dictionaries:

In [ ]:
# Raw streaming - hard to follow!
for chunk in agent.stream(
    dict(messages=["What files are in the current directory?"]), 
    config=dict(configurable=dict(thread_id="raw-demo")),
    stream_mode="updates"
):
    print(chunk)
    print("---")

---
## After: JupyterDisplay

Same stream, but with clean, live-updating panels:

In [ ]:
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

In [ ]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["What files are in the current directory?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

---
## Key Features

### 1. Tool Lifecycle Tracking

Tools show their status in real-time:
- `⏳ ...` - Running
- `✅ OK` - Success (with execution time)
- `❌ ERR` - Error

### 2. Interrupt Handling

When the agent needs approval (like for `bash` commands), you get a prominent red panel showing:
- The tool requesting approval
- The arguments it wants to use
- Allowed decisions (approve/reject/edit)

### 3. Content Accumulation

Assistant responses stream smoothly in a blue panel instead of chunk-by-chunk spam.

---
## Manual Event Processing

For more control, you can process events individually:

In [ ]:
from langgraph_stream_parser import StreamParser
from langgraph_stream_parser.events import InterruptEvent, ContentEvent, ToolCallStartEvent

parser = StreamParser()
display = JupyterDisplay()

for event in parser.parse(
    agent.stream(
        dict(messages=["Count Python files in this directory"]),
        config=dict(configurable=dict(thread_id="manual-demo")),
        stream_mode="updates"
    )
):
    display.update(event)
    
    # You can also handle events programmatically
    if isinstance(event, InterruptEvent):
        print(f"\n[Interrupt detected - {len(event.action_requests)} action(s) need approval]")

---
## Configuration Options

In [ ]:
# Customize the display
display = JupyterDisplay(
    show_timestamps=False,     # Show/hide timestamps
    show_tool_args=True,       # Show tool arguments in the status table
    max_content_preview=200,   # Max characters for extraction previews
)

---
## Summary

| Raw Streaming | JupyterDisplay |
|---------------|----------------|
| Nested dicts flood output | Clean, organized panels |
| Tool status buried in data | Visual tool lifecycle (⏳→✅/❌) |
| Interrupts easy to miss | Prominent red interrupt panel |
| Content comes in chunks | Smooth content accumulation |
| No timing information | Execution time per tool |